🧩 1. Instalación de librerías necesarias

In [27]:
!pip install xarray rioxarray netcdf4 netCDF4 pandas folium scipy


📥 2. Carga de archivos .nc

In [28]:
import xarray as xr

# Cargar radiación solar (desde ERA5, resolución regular)
ds_radiacion = xr.open_dataset("datos_netcdf/radiacion_solar_era5_2024_01.nc", engine="netcdf4")
print("📦 Dataset de radiación solar:")
print(ds_radiacion)

# Cargar temperatura, punto de rocío y precipitación (desde ERA5-Land, alta resolución)
# (ajusta el nombre si el archivo no se llama exactamente así dentro del zip descomprimido)
ds_meteo = xr.open_dataset("datos_era5_land/data_0.nc", engine="netcdf4")
print("\n📦 Dataset de temperatura, rocío y precipitación:")
print(ds_meteo)

📦 Dataset de radiación solar:
<xarray.Dataset> Size: 36kB
Dimensions:     (valid_time: 744, latitude: 3, longitude: 2)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 6kB 2024-01-01 ... 2024-01-31T23:...
  * latitude    (latitude) float64 24B 5.3 5.05 4.8
  * longitude   (longitude) float64 16B -75.75 -75.5
    expver      (valid_time) <U4 12kB ...
Data variables:
    ssrd        (valid_time, latitude, longitude) float32 18kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-06-24T17:41 GRIB to CDM+CF via cfgrib-0.9.1...

📦 Dataset de temperatura, rocío y precipitación:
<xarray.Dataset> Size: 330kB
Dimensions:     (valid_time: 744, latitude: 7, longitude: 5)
Coordinates:
    number      int64

🧮 3. Procesamiento a DataFrame

## 🌤️ Interpolación espacial de la radiación solar para datos meteorológicos

### Objetivo

Al trabajar con datos meteorológicos provenientes de dos fuentes diferentes:

- **ERA5 (resolución baja):** contiene datos de **radiación solar** (`ssrd`) en una malla de baja densidad espacial (solo 3 latitudes × 2 longitudes).
- **ERA5-Land (resolución alta):** contiene datos de **temperatura**, **punto de rocío** y **precipitación** en una malla más densa (7 latitudes × 5 longitudes).

El reto es unir ambas fuentes para tener todas las variables en la misma resolución espacial. Para lograrlo, **asignamos a cada punto de ERA5-Land el valor de radiación solar más cercano disponible** desde ERA5, usando **interpolación por vecino más cercano**.

---

### ¿Qué hace el código?

1. **Extrae las coordenadas disponibles** de cada fuente:
   - `coords_era5`: coordenadas donde sí se mide la radiación solar.
   - `coords_land`: coordenadas donde se miden las otras variables, pero falta la radiación.

2. **Crea un árbol espacial (k-d tree)** usando `scipy.spatial.cKDTree` para buscar eficientemente el vecino más cercano en el espacio geográfico.

3. **Calcula el índice del punto más cercano** en ERA5 para cada punto de ERA5-Land.

4. **Genera un DataFrame de emparejamiento** entre puntos de ERA5-Land y sus vecinos más cercanos en ERA5.

5. **Asocia los valores de radiación solar reales** a los nuevos puntos de alta resolución.

6. **Renombra columnas y une ambos conjuntos de datos** en un único DataFrame, ahora con la radiación solar interpolada espacialmente.

---

### ¿Por qué usar interpolación por vecino más cercano?

Este método es útil cuando:

- Se necesita una solución rápida y sin muchos supuestos.
- Las distancias entre puntos no son grandes.
- No se requiere suavizado espacial ni gradientes continuos.

También es ideal para mantener consistencia en estudios que requieren granularidad homogénea como modelos predictivos o mapas diarios con alta resolución espacial.

---

### Resultado

Al final, el DataFrame `df_comb` tiene las siguientes variables:

- `temperatura_2m_C`
- `punto_rocio_2m_C`
- `precipitacion_total_mm`
- `radiacion_solar_whm2` ← ahora interpolada

Todas disponibles para el mismo conjunto de coordenadas y fechas, listos para análisis, visualización y modelado.



In [29]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

# Asegúrate de que las coordenadas estén bien nombradas
df_land["valid_time"] = pd.to_datetime(df_land["valid_time"])
df_radiacion["valid_time"] = pd.to_datetime(df_radiacion["valid_time"])

# Construimos KDTree con puntos de radiación solar
coords_radiacion = df_radiacion[["latitude", "longitude"]].drop_duplicates().to_numpy()
tree = cKDTree(coords_radiacion)

# Para cada punto de df_land, encontramos el más cercano
coords_land = df_land[["latitude", "longitude"]].to_numpy()
_, idx = tree.query(coords_land, k=1)

# Asignamos las coordenadas de radiación solar más cercanas
df_land["lat_radiacion"] = coords_radiacion[idx][:, 0]
df_land["lon_radiacion"] = coords_radiacion[idx][:, 1]

# Merge por tiempo y coordenadas de radiación emparejadas
df_merge = pd.merge(
    df_land,
    df_radiacion,
    left_on=["valid_time", "lat_radiacion", "lon_radiacion"],
    right_on=["valid_time", "latitude", "longitude"],
    how="left",
    suffixes=("", "_rad")
)

# Renombrar y convertir unidades
df_merge.rename(columns={
    "valid_time": "fecha",
    "t2m": "temperatura_2m_C",
    "d2m": "punto_rocio_2m_C",
    "tp": "precipitacion_total_mm",
    "ssrd": "radiacion_solar_whm2"
}, inplace=True)

df_merge["temperatura_2m_C"] -= 273.15
df_merge["punto_rocio_2m_C"] -= 273.15
df_merge["precipitacion_total_mm"] *= 1000
df_merge["radiacion_solar_whm2"] /= 3600
df_merge["fecha"] = pd.to_datetime(df_merge["fecha"]).dt.date

# Agregación diaria por punto
columnas = [
    "temperatura_2m_C", "punto_rocio_2m_C",
    "precipitacion_total_mm", "radiacion_solar_whm2"
]
df_diario = df_merge.groupby(["latitude", "longitude", "fecha"])[columnas].mean().reset_index()
df_diario.rename(columns={"latitude": "latitud", "longitude": "longitud"}, inplace=True)


🧮 4. Generación de características avanzadas

In [30]:
# === 3. Calcular variables adicionales ===
def add_moving_features(df, col, nombre_base):
    for dias in [4, 7, 14, 21, 28]:
        df[f"{nombre_base}_prom_{dias}d"] = (
            df.groupby(["latitud", "longitud"])[col]
            .transform(lambda x: x.rolling(window=dias, min_periods=1).mean())
        )
        df[f"{nombre_base}_acum_{dias}d"] = (
            df.groupby(["latitud", "longitud"])[col]
            .transform(lambda x: x.rolling(window=dias, min_periods=1).sum())
        )

add_moving_features(df_diario, "radiacion_solar_whm2", "radiacion_solar")
add_moving_features(df_diario, "temperatura_2m_C", "temperatura_2m")

for dias in [1, 2, 3, 4, 5]:
    df_diario[f"precipitacion_acum_{dias}d"] = (
        df_diario.groupby(["latitud", "longitud"])["precipitacion_total_mm"]
        .transform(lambda x: x.rolling(window=dias, min_periods=1).sum())
    )

# Resultado final
df_final = df_diario.copy()
print("✅ DataFrame listo para análisis/modelado")
print(df_final.head())


✅ DataFrame listo para análisis/modelado
   latitud  longitud       fecha  temperatura_2m_C  punto_rocio_2m_C  \
0      4.8    -75.75  2024-01-01         18.165018         16.879770   
1      4.8    -75.75  2024-01-02         18.208475         16.801636   
2      4.8    -75.75  2024-01-03         18.738922         16.919413   
3      4.8    -75.75  2024-01-04         18.144876         16.242483   
4      4.8    -75.75  2024-01-05         17.854889         15.632568   

   precipitacion_total_mm  radiacion_solar_whm2  radiacion_solar_prom_4d  \
0               26.812355            200.222336               200.222336   
1               11.881217            210.279526               205.250931   
2               34.142059            234.838638               215.113500   
3               17.981943            247.210007               223.137627   
4               12.527724            228.971573               230.324936   

   radiacion_solar_acum_4d  radiacion_solar_prom_7d  ...  \
0        

🧮 5. Exportar a CSV

In [31]:
# === 4. Guardar como CSV ===
output_path = "datos_meteorologicos_2024_01.csv"
df_final.to_csv(output_path, index=False)

print(f"✅ Archivo CSV guardado correctamente en: {output_path}")


✅ Archivo CSV guardado correctamente en: datos_meteorologicos_2024_01.csv


| Variable                 | Descripción                                                      |
| ------------------------ | ---------------------------------------------------------------- |
| `latitud`                | Latitud del píxel o punto geográfico (°)                         |
| `longitud`               | Longitud del píxel o punto geográfico (°)                        |
| `fecha`                  | Fecha de observación (AAAA-MM-DD)                                |
| `temperatura_2m_C`       | Temperatura promedio diaria del aire a 2 metros (°C)             |
| `punto_rocio_2m_C`       | Temperatura promedio diaria del punto de rocío a 2 metros (°C)   |
| `precipitacion_total_mm` | Precipitación total diaria (mm)                                  |
| `radiacion_solar_whm2`   | Radiación solar superficial descendente acumulada diaria (Wh/m²) |

| Variables derivadas        | Descripción                                        |
| -------------------------- | -------------------------------------------------- |
| `radiacion_solar_prom_4d`  | Promedio de radiación solar en los últimos 4 días  |
| `radiacion_solar_acum_4d`  | Acumulado de radiación solar en los últimos 4 días |
| `radiacion_solar_prom_7d`  | Promedio de 7 días                                 |
| `radiacion_solar_acum_7d`  | Acumulado de 7 días                                |
| `radiacion_solar_prom_14d` | Promedio de 14 días                                |
| `radiacion_solar_acum_14d` | Acumulado de 14 días                               |
| `radiacion_solar_prom_21d` | Promedio de 21 días                                |
| `radiacion_solar_acum_21d` | Acumulado de 21 días                               |
| `radiacion_solar_prom_28d` | Promedio de 28 días                                |
| `radiacion_solar_acum_28d` | Acumulado de 28 días                               |
| `temperatura_2m_prom_4d`   | Promedio de temperatura de los últimos 4 días |
| `temperatura_2m_acum_4d`   | Suma de temperatura de los últimos 4 días     |
| `temperatura_2m_prom_7d`   | Promedio de 7 días                            |
| `temperatura_2m_acum_7d`   | Suma de 7 días                                |
| `temperatura_2m_prom_14d`  | Promedio de 14 días                           |
| `temperatura_2m_acum_14d`  | Suma de 14 días                               |
| `temperatura_2m_prom_21d`  | Promedio de 21 días                           |
| `temperatura_2m_acum_21d`  | Suma de 21 días                               |
| `temperatura_2m_prom_28d`  | Promedio de 28 días                           |
| `temperatura_2m_acum_28d`  | Suma de 28 días                               |
| `precipitacion_acum_1d`    | Precipitación acumulada de 1 día |
| `precipitacion_acum_2d`    | Acumulado de 2 días              |
| `precipitacion_acum_3d`    | Acumulado de 3 días              |
| `precipitacion_acum_4d`    | Acumulado de 4 días              |
| `precipitacion_acum_5d`    | Acumulado de 5 días              |


In [32]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm

# 1. Cargar el DataFrame final desde CSV
df = pd.read_csv("/workspaces/hackatonfnc/datos_meteorologicos_2024_01.csv")

# 2. Filtrar una fecha específica
fecha_mostrar = "2024-01-15"
df_dia = df[df["fecha"] == fecha_mostrar]

# 3. Crear mapa base centrado en Caldas
bbox = [-75.75, 4.8, -75.35, 5.4]
m = folium.Map(location=[(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2], zoom_start=10)

# 4. Variables a graficar (nombre en CSV : nombre para leyenda)
variables = {
    "radiacion_solar_whm2": "Radiación solar (Wh/m²)",
    "temperatura_2m_C": "Temperatura 2m (°C)",
    "precipitacion_total_mm": "Precipitación total (mm)",
    "punto_rocio_2m_C": "Punto de rocío 2m (°C)"
}

# 5. Crear capas por variable
for var, nombre in variables.items():
    capa = folium.FeatureGroup(name=nombre, show=False)

    valores = df_dia[var]
    colormap = cm.linear.YlOrRd_09.scale(valores.min(), valores.max())
    colormap.caption = nombre
    colormap.add_to(m)

    for _, row in df_dia.iterrows():
        valor = row[var]
        color = colormap(valor)

        # Tooltip flotante (rápido)
        tooltip = f"{nombre}: {valor:.2f}"

        # Popup más completo
        popup = folium.Popup(f"""
        <b>Fecha:</b> {row['fecha']}<br>
        <b>Latitud:</b> {row['latitud']:.4f}<br>
        <b>Longitud:</b> {row['longitud']:.4f}<br>
        <b>{nombre}:</b> {valor:.2f}
        """, max_width=250)

        folium.CircleMarker(
            location=[row["latitud"], row["longitud"]],
            radius=6,
            color=color,
            fill=True,
            fill_opacity=0.9,
            tooltip=tooltip,
            popup=popup
        ).add_to(capa)

    capa.add_to(m)

# 6. Bounding box (área de interés)
folium.Rectangle(
    bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],
    color="blue", fill=False, weight=2, tooltip="Área de interés"
).add_to(m)

# 7. Controles de capa
folium.LayerControl(collapsed=False).add_to(m)

# 8. Mostrar mapa
m


In [33]:
print(df_radiacion["valid_time"].nunique())  # 744 fechas
print(df_radiacion.groupby("valid_time")["latitude"].nunique())  # ¿cuántas coordenadas?


744
valid_time
2024-01-01 00:00:00    3
2024-01-01 01:00:00    3
2024-01-01 02:00:00    3
2024-01-01 03:00:00    3
2024-01-01 04:00:00    3
                      ..
2024-01-31 19:00:00    3
2024-01-31 20:00:00    3
2024-01-31 21:00:00    3
2024-01-31 22:00:00    3
2024-01-31 23:00:00    3
Name: latitude, Length: 744, dtype: int64
